# Découverte des autoencoders

Après des tentatives de système de recommendation de film en se basant sur des regressions, on va tenter une nouvelle approche en utilisant les autoencoders qui sont, semble-t-il, particulièrement adaptés à ce genre de problème.

## Jeu de données

Les notes ont été parsées depuis un site de notation, mais ca n'est pas le sujet ici. Les données sont stockées dans un fichier dump.

In [1]:
import pickle
outfile = open('results.dmp', 'rb')
datas = pickle.load(outfile)
outfile.close()

On a fait au plus simple, datas est un dictionnaire donc les clés sont les ids des films et les valeurs sont des dictionnaires contenant les infos du film (titre, année, acteurs, ... et les différentes notes).

In [2]:
datas[368836]

{'id': 368836,
 'title': 'Brisby et le Secret de NIMH',
 'year': '1982',
 'note_globale': 7.6,
 'url': 'https://www.senscritique.com/film/Brisby_et_le_Secret_de_NIMH/368836',
 'image': 'https://media.senscritique.com/media/000016246779/90/Brisby_et_le_Secret_de_NIMH.jpg',
 'duration': '1 h 22 min',
 'release_date': '16 juillet 1982',
 'genres': 'Animation,drame,fantastique,science-fiction',
 'categorie': "Long-métrage d'animation",
 'directors': 'Don Bluth',
 'actors': 'Derek Jacobi,Elizabeth Hartman,Arthur Malet',
 'note_A': None,
 'note_B': 8.0,
 'note_C': 8.0,
 ...
 'note_ZZZZ': 8.0}

Pour chaque film, on a donc une liste de notes (les clés préfixées par note). On va devoir réorganiser ces données pour les fournir en entrée de notre autoencoder. L'idée est de créer tableau numpy avec les colonnes correspondant au film, chaque ligne correspondant à un utilisateur.

## Préparation des données

On définit une fonction qui parcourt les données et créé le tableau correspondant.
NB : je n'utilise pas de dataframe pandas comme habituellement car les performances sont catastrophiques.

In [3]:
import numpy as np


def formatData(data):
    """Transform the dictionnary data into numpy array for machine learning
    
    """
    users = []
    for item in data.values():
        for note in item.keys():
            if "note_" in note and note != 'note_globale' and item[note] is not None:
                user = note.replace('note_','')
                if user not in users:
                    users.append(user)
                
    films = list(data.keys())

    nb_movies = len(films)
    nb_users = len(users)


    films_id_to_num = {}
    users_name_to_num = {}

    for i in range(0,len(films)):
        films_id_to_num[films[i]] = i

    for i in range(0,len(users)):
        users_name_to_num[users[i]] = i

    X = np.zeros([nb_users, nb_movies])


    for item in data.values():
        for note in item.keys():
            if "note_" in note and note != 'note_globale' and item[note] is not None:
                num_line = users_name_to_num[note.replace('note_','')]
                num_col = films_id_to_num[item['id']]
                X[num_line,num_col] = item[note]
    
    return X, films, users

In [7]:
X, films, users = formatData(datas)

On a donc notre tableau X et les 2 tableaux permettant de faire respectivement le lien entre l'id du film et la colonne correspondante et le nom du user et la ligne correspondante.

In [8]:
X.shape

(794, 86466)

86466 films... c'est beaucoup. Voir même ridicule. On a récupéré des données qui n'ont pas vraiment de sens, le film qui n'est noté que par un ou deux utilisateur n'a pas vraiment vocation à être analysé. On écrit donc une fonction pour faire le ménage et ne garder que les films qui ont au moins 250 notes.

In [9]:
def cleanData(X, films, min_num_notes = 150):
    
    # Count number of film to keep (for performance better than rebuild the array)
    nb_users = len(X[:,0])
    nb_movies = len(X[0,:])
    
    nb_movies_to_keep = 0
    for i in range(nb_movies):
        if np.size(np.nonzero(X[:,i])) > min_num_notes:
            nb_movies_to_keep += 1

    X_filtered = np.zeros([nb_users, nb_movies_to_keep])
    
    
    # Filling the array with data
    films_to_keep = []
    current=0

    for i in range(nb_movies):
        if np.size(np.nonzero(X[:,i])) > min_num_notes:
            X_filtered[:,current] = X[:,i]
            current += 1
            films_to_keep.append(films[i])
    
    X = X_filtered
    films = films_to_keep
    
    return X, films

In [10]:
X, films =  cleanData(X, films, 250)

In [11]:
X.shape

(794, 1199)

1200 films, c'est plus raisonnable et suffisant pour un premier test. Il faudrait peut-être également supprimer les utilisateurs qui n'ont pas noté suffisamment de films (à réfléchir si cela a un sens).

Notre tableau est prêt. Il nous reste à construire les dictionnaires pour retrouver les films/utilisateurs en fonction des colonnes/lignes.

In [12]:
def buildReverseList(liste):
    new_liste = {}
    for i in range(len(liste)):
        new_liste[liste[i]] = i
    return new_liste

In [13]:
films_id_to_num = buildReverseList(films)
users_name_to_num = buildReverseList(users)

In [14]:
nb_movies = len(films)
nb_users = len(users)

## Création de l'autoencoder

On va réutiliser un morceau de code qu'on retrouve un peu partout et qui utilise pytorch pour créer rapidement un autoencoder. La difficulté ici tient au fait qu'il ne faut pas rétropropager l'erreur lorsque le film n'a pas été vu par l'utilisateur sinon, on pourrait tout simplement utiliser les librairies standard sans rien écrire nous même.

In [15]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [16]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

sae est donc notre instance d'autoencoder. Rien de particulier jusque là. On utilise les bibliothèque sklearn pour créer le training set et le test set (à voir si c'est aussi simple avec pytorch)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size = 0.2, random_state = 0)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(X_train)
test_set = torch.FloatTensor(X_test)

On entraine notre réseau. C'est ici la grosse difficulté parce qu'il faut customiser le calcul de l'erreur.

In [18]:
# Training the SAE
nb_users = X_train[:,0].size
nb_epoch = 50
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.742316641815268
epoch: 2 loss: 1.5375415669353552
epoch: 3 loss: 1.5369872627804497
epoch: 4 loss: 1.5365561653105855
epoch: 5 loss: 1.5367702856344025
epoch: 6 loss: 1.5340909672523577
epoch: 7 loss: 1.5349719458431026
epoch: 8 loss: 1.5349076357637743
epoch: 9 loss: 1.534917788142103
epoch: 10 loss: 1.5323347102974552
epoch: 11 loss: 1.5329856863953406
epoch: 12 loss: 1.530052840408911
epoch: 13 loss: 1.5336292175430775
epoch: 14 loss: 1.53366980642044
epoch: 15 loss: 1.5328542101868838
epoch: 16 loss: 1.5323147280520493
epoch: 17 loss: 1.53183271296583
epoch: 18 loss: 1.5314819879819732
epoch: 19 loss: 1.531166345888099
epoch: 20 loss: 1.5310304305879014
epoch: 21 loss: 1.5309771268368306
epoch: 22 loss: 1.5308580780685241
epoch: 23 loss: 1.5309373718648702
epoch: 24 loss: 1.5316064557515274
epoch: 25 loss: 1.5298965763314283
epoch: 26 loss: 1.5303870712737182
epoch: 27 loss: 1.531016652644922
epoch: 28 loss: 1.5310395633049427
epoch: 29 loss: 1.5310833946182556
epo

On laisse tous les paramètres par défaut... c'est juste un premier jet pour découvrir les autoencoders et on teste le résultat sur notre test set :

In [19]:
# Testing the SAE
nb_users = X_test[:,0].size
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(test_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 1.5139577117563043


OK l'erreur est sensiblement la même que sur le training set, ca parait bon (voir même trop bon... à réfléchir).

## Utilisation de l'autoencoder

Essayons maintenant de faire des recommendations (pour moi tant qu'à faire). Pour cela, il faut mettre en entrée de notre réseau l'ensemble des notes de l'utilisateur.

In [20]:
# Using the SAE
all_set = torch.FloatTensor(X)
input = Variable(all_set[users_name_to_num['hfred1982']]).unsqueeze(0)
output = sae(input)

La variable output contient donc les notes prédites par notre réseau pour les n films étudiés. Première question : est-ce qu'on fait mieux qu'un estimateur simpliste qui prendrait simplement la note du public comme prédiction ? Le note du public étant disponible dans nos données, c'est facile à vérifier.

In [23]:
error = 0
error_note_globale = 0
nb_films = 0
for i in range(nb_movies):
    if X[users_name_to_num['hfred1982']][i] != 0:
        error += (X[users_name_to_num['hfred1982']][i] - output.data[0][i])**2
        error_note_globale += (X[users_name_to_num['hfred1982']][i] - datas[films[i]]['note_globale'])**2
        nb_films +=1 
        
print("Erreur de l'autoencoder : "+str(np.sqrt(error/nb_films)))
print("Erreur de la note globale : "+str(np.sqrt(error_note_globale/nb_films)))

Erreur de l'autoencoder : 1.3939345311016689
Erreur de la note globale : 1.4317210178265514


On fait mieux ouf ... Allons-y alors et faisons notre première recommendation de film. Auparavant, on va nettoyer notre sortie, en mettant 0 aux films déjà vus.

In [24]:
prediction = []
for i in range(0,nb_movies):
    if X[users_name_to_num['hfred1982']][i] == 0:
        prediction.append(output.data[0][i])
    else:
        prediction.append(0)

Et le meilleur film prédit est :

In [25]:
# meilleur film prédit
datas[films[np.argmax(np.array(prediction))]]['title']

"L'Aurore"

Ca semble crédible même si a priori je ne vois pas pourquoi ce film me serait proposé plutôt qu'un autre. Une question à vérifier tout de même : est-ce que ca n'est pas tout simplement le film qui a la meilleure moyenne parmi les notes récupérées ? Auquel cas, l'autoencoder ne sert pas à grand chose.

In [27]:
# On calcule la moyenne de chaque film sur le test set (on met 0 si on l'a déjà vu)
moy_films = []
for i in range(nb_movies):
    if X[users_name_to_num['hfred1982']][i] == 0:
        moy_films.append(sum(X_test[:,i])/np.count_nonzero(X_test[:,i]))
    else:
        moy_films.append(0)

# On regarde quel film a la meilleure moyenne
datas[films[np.argmax(moy_films)]]['title']

'Chantons sous la pluie'

Sauvé ! On ne me propose pas bêtement le film avec la meilleure moyenne, c'est donc un conseil "personalisé".

Essayons maintenant de voir un peu plus en détail comment fonctionne l'autoencoder : d'une certaine façon, l'autoencoder va détecter automatiquement certaines features des films qui vont permettre d'expliquer leur note. Pour voir cela, on peut essayer de regarder le premier layer et de voir quels sont les poids associés à chaque film.

In [30]:
sae.fc1.weight

Parameter containing:
 1.0979e-01 -1.3440e-02 -9.0807e-02  ...  -8.7257e-05  1.5656e-02  1.2193e-02
-2.0495e-02  7.6809e-02  3.7586e-02  ...   3.9110e-02  5.9315e-02  5.8146e-02
-1.2941e-01 -3.6356e-01 -1.1999e-01  ...   7.4995e-02 -6.1159e-02  9.4228e-02
                ...                   ⋱                   ...                
-1.5531e-02 -2.6397e-01 -1.5148e-02  ...   6.3949e-02  5.2527e-02  6.5169e-02
-3.2325e-01 -1.8299e-01 -4.0070e-01  ...   6.1947e-01  6.3777e-01  6.3676e-01
-1.4615e-02 -4.8421e-02 -6.2948e-02  ...   8.2534e-03 -9.4053e-03  2.8250e-03
[torch.FloatTensor of size 20x1199]

Les poids sont stockés dans un tenseur 20x(nb_films). On peut donc facilement pour chaque noeud regarder quels sont les films qui ont le poid le plus fort associé.

In [36]:
print("Les 10 films au poids le plus important sur le noeud 1 :")
for i in sae.fc1.weight[1].sort()[1][-10:].data:
    print("\t"+datas[films[i]]['title'])

Les 10 films au poids le plus important sur le noeud 1 :
	Rox et Rouky
	Un long dimanche de fiançailles
	Star Wars : Les Derniers Jedi
	Aladdin
	La Petite Sirène
	Star Wars : Épisode III - La Revanche des Sith
	La Liste de Schindler
	Star Wars : Le Réveil de la Force
	Le Roi Lion
	La Belle et la Bête


Difficile de voir un critère qui serait valable pour tous ces films. Néanmoins, on peut remarquer la présence de 5 dessins animés et de 3 Stars Wars. C'est intéressant car on peut difficilement croire que c'est le hasard qui a regroupé ces films alors que l'autoencoder lui n'a aucun moyen de savoir que ces films appartiennent à la même franchise. On peut donc penser qu'il a donc bien "détecté" une feature qui est commune à tous (même si encore une fois, elle est difficile à expliciter). Regardons sur un autre noeud pour confirmer.

In [41]:
print("Les 10 films au poids le plus important sur le noeud 2 :")
for i in sae.fc1.weight[2].sort()[1][-10:].data:
    print("\t"+datas[films[i]]['title'])

Les 10 films au poids le plus important sur le noeud 2 :
	10 Cloverfield Lane
	Pacific Rim
	Taken
	Inception
	Spider-Man : New Generation
	Coco
	Captain America : Civil War
	Doctor Strange
	Avengers : Endgame
	La Forme de l'eau


Encore une fois, pas de thème véritablement unificateur, mais tout de même 4 films Marvels regroupés. C'est bon signe.
On va sauvegarder notre modèle pour repartir sur la même analyse la prochaine fois.

In [43]:
torch.save(sae.state_dict(), 'sae.pth')

Voilà pour un premier jet, reste bien entendu à tuner les hyperparamètres et voir si on peut en tirer plus d'enseignement.